# Generate Power Functional Connectivity
This script performs further processing on the data by generating the functional connectivities from the preprocessed data. We take in data from the final output of AA in the `aamod_firstlevel_modelestimate_saveresids_00001` folder and generate functional connectivity from the NIfTI files using the Power et al. brain atlas composed of 264 ROIs.

I'm following the guide here: <https://nilearn.github.io/auto_examples/03_connectivity/plot_sphere_based_connectome.html>

## Set up imports

In [2]:
from os import listdir, makedirs
from os.path import isfile, join, exists
import time

import numpy as np
from nilearn import image
from common.paths import POWER_FC
from common.power_atlas import generate_power_fc_matrix

## Set up input and output

In [3]:
aa_folder = 'BioBank_Analysis_All/'
saveresids_folder = 'aamod_firstlevel_modelestimate_saveresids_00001/'
python_fc_folder = 'python_power_fc'

## Generate FCs

In [ ]:
for age in range(7, 8):
    print("Processing age:", age)
#     age_dir = f'/imaging/owenlab/klyons/fMRI/CBS_DevCog/BioBankData/Age{age}Data/'
    age_dir = '/imaging3/owenlab/wilson/Healthy'

    input_path = join(age_dir, aa_folder, saveresids_folder)
    output_path = join(POWER_FC, 'Healthy')

    # Grab all subjects
    subjects = [folder for folder in listdir(input_path) if folder.startswith("sub-")]

    for subject in subjects:
        print("Processing subject:", subject)
        start_time = time.time()
        subject_input_path = join(input_path, subject)
        subject_output_path = join(output_path, subject)

        # Check if we've already generated the FC for this subject
        if exists(subject_output_path):
            print("  Skipping subject due to already generated FC.")
            continue

        subject_niftis = [join(subject_input_path, f) for f in listdir(subject_input_path) 
                          if isfile(join(subject_input_path, f)) and f.startswith('ResI')]

        if len(subject_niftis) != 745:
            print("  Incomplete subject niftis:", len(subject_niftis))
            continue

        subject_concat_image = image.concat_imgs(subject_niftis)
        subject_corr_matrix = generate_power_fc_matrix()

        makedirs(subject_output_path)
        subject_corr_filename = join(subject_output_path, 'power_fc.npy')
        np.save(subject_corr_filename, subject_corr_matrix)
        print("  Time elapsed:", time.time() - start_time)